# 02_baseline_tfidf — Binario A/D

**Objetivo:** baseline clásico **robusto a ruido** (typos/transcripción) con **char TF‑IDF (3–5)** + **SVM (LinearSVC)**.  
**Justificación:** los n‑gramas de caracteres capturan patrones ortográficos aun con errores; es un buen contrapunto al enfoque rule‑based.


In [1]:
# ===============================================================
# Setup: Paths, Imports, y Utilidades Compartidas
# ===============================================================

from pathlib import Path
import pandas as pd
import re, unicodedata, os

# Intentar importar utilidades compartidas
try:
    import sys
    sys.path.insert(0, str(Path.cwd()))
    from utils_shared import setup_paths, guess_text_col, guess_label_col, normalize_label
    print("[INFO] Utilizando utils_shared.py")
    
    # Setup de paths centralizado
    paths = setup_paths()
    BASE_PATH = paths['BASE_PATH']
    DATA_PATH = paths['DATA_PATH']
    SPLITS_PATH = paths['SPLITS_PATH']
    
    # Usar funciones centralizadas
    _guess_text_col = guess_text_col
    _guess_label_col = guess_label_col
    _norm_label_bin = normalize_label
    
except ImportError:
    print("[WARNING] utils_shared.py no encontrado, usando funciones locales")
    
    # Setup manual de paths
    BASE_PATH = Path.cwd()
    if BASE_PATH.name == "notebooks":
        BASE_PATH = BASE_PATH.parent
    
    DATA_PATH = BASE_PATH / "data"
    SPLITS_PATH = DATA_PATH / "splits"
    
    DATA_PATH.mkdir(exist_ok=True)
    
    # Funciones helper locales
    def _guess_text_col(df):
        for c in ["texto", "text", "comment", "comentario"]:
            if c in df.columns:
                return c
        return df.columns[0]
    
    def _guess_label_col(df):
        for c in ["etiqueta", "label", "category"]:
            if c in df.columns:
                return c
        return df.columns[1] if len(df.columns) > 1 else df.columns[-1]
    
    def _norm_label_bin(s):
        if pd.isna(s): 
            return ""
        s = str(s).strip().lower()
        s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
        return {'depresivo': 'depresion'}.get(s, s)

# Validar existencia de splits
if not SPLITS_PATH.exists():
    raise FileNotFoundError(
        f"[ERROR] Splits no encontrados en {SPLITS_PATH}\n"
        f"        Debes ejecutar primero: 02_create_splits.ipynb"
    )

print("Paths configurados:")
print(f"  BASE_PATH:   {BASE_PATH}")
print(f"  DATA_PATH:   {DATA_PATH}")
print(f"  SPLITS_PATH: {SPLITS_PATH}")

# Columnas esperadas en dataset_base.csv
TEXT_COL = "texto"
LABEL_COL = "etiqueta"

[INFO] Utilizando utils_shared.py
Paths configurados:
  BASE_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay
  DATA_PATH:   /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data
  SPLITS_PATH: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits


## 1) Carga y preprocesamiento **agresivo** (pensado para ML clásico)

In [2]:
# ===============================================================
# CARGA DE DATOS - PATIENT-LEVEL SPLIT (SIN LEAKAGE)
# ===============================================================
# IMPORTANTE: Este baseline usa splits generados por 02_create_splits.ipynb
#
# Estrategia de split:
#   - Por PACIENTES (no por casos/consultas)
#   - 72 pacientes train / 18 pacientes val
#   - 0% overlap (sin data leakage)
#
# ¿Por qué patient-level?
#   Dataset tiene estructura longitudinal: 90 pacientes × 35 consultas promedio
#   Split por casos → 100% pacientes en train Y val (leakage total)
#   Split por pacientes → 0% overlap (generaliza a pacientes nuevos)
#
# Ver detalles en: ESTRATEGIA_SPLIT_PACIENTES.md

import pandas as pd, re, unicodedata

# Cargar splits unificados desde 02_create_splits.ipynb
print("="*60)
print("CARGA DE SPLITS (PATIENT-LEVEL)")
print("="*60)

dataset_base = pd.read_csv(SPLITS_PATH / 'dataset_base.csv')
train_indices = pd.read_csv(SPLITS_PATH / 'train_indices.csv')['row_id'].values
dev_indices = pd.read_csv(SPLITS_PATH / 'dev_indices.csv')['row_id'].values

print(f"✓ Dataset base: {len(dataset_base)} casos")
print(f"✓ Train indices: {len(train_indices)} casos")
print(f"✓ Dev indices: {len(dev_indices)} casos")

text_col = _guess_text_col(dataset_base)
label_col = _guess_label_col(dataset_base)

# ===============================================================
# PREPROCESAMIENTO AGRESIVO (máxima normalización para TF-IDF)
# ===============================================================
# Estrategia: Máxima normalización (lowercase, sin tildes, sin símbolos)
# - Reduce vocabulario y mejora robustez ante typos
# - Marca negaciones: "no tengo" → "no_tengo" (feature diferenciado)
# - Char n-grams (3-5): robustos a variantes ortográficas
#
# Comparación: Rule-based conserva tildes, BETO solo tokeniza, TF-IDF normaliza todo

RE_MULTI = re.compile(r'(.)\1{2,}')  # Detecta 3+ letras repetidas

def clean_text_ml(s: str) -> str:
    """
    Limpieza AGRESIVA para TF-IDF (máxima normalización para robustez).
    
    Aplica:
    - Lowercase completo
    - Normalización NFC + eliminación de tildes
    - Colapso de alargamientos
    - Eliminación de símbolos especiales (preserva puntuación básica)
    - Marca negaciones simples ("no X" → "no_X")
    """
    if pd.isna(s):
        return ""
    
    s = str(s).lower().strip()           # Lowercase total
    s = unicodedata.normalize("NFC", s)  # Normaliza tildes
    s = RE_MULTI.sub(r'\1\1', s)         # Colapsa alargamientos
    
    # Elimina símbolos especiales (preserva letras, números, puntuación básica)
    s = re.sub(r"[^a-z0-9áéíóúüñ\s.,!?:/\-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()   # Normaliza espacios
    
    # Marca negaciones simples: "no tengo" → "no_tengo"
    s = re.sub(r"\bno\s+([a-záéíóúüñ]{2,})", r"no_\1", s)
    
    return s

dataset_base['texto_ml'] = dataset_base[text_col].map(clean_text_ml)

# Filtrar por índices (patient-level split)
df_train = dataset_base[dataset_base['row_id'].isin(train_indices)].copy()
df_val = dataset_base[dataset_base['row_id'].isin(dev_indices)].copy()

X_train, y_train = df_train['texto_ml'], df_train[label_col]
X_val, y_val = df_val['texto_ml'], df_val[label_col]

print(f"\n[INFO] Split aplicado (patient-level):")
print(f"  Train: {len(X_train)} casos")
print(f"  Val:   {len(X_val)} casos")
print(f"\n[INFO] Distribución train: {dict(y_train.value_counts())}")
print(f"[INFO] Distribución val: {dict(y_val.value_counts())}")
print("\n  RECORDATORIO: Estos splits eliminan leakage (pacientes disjuntos)")
print("   Métricas serán más conservadoras pero generalizan mejor.")

CARGA DE SPLITS (PATIENT-LEVEL)
✓ Dataset base: 3127 casos
✓ Train indices: 1849 casos
✓ Dev indices: 641 casos

[INFO] Split aplicado (patient-level):
  Train: 1849 casos
  Val:   641 casos

[INFO] Distribución train: {'depresion': 1270, 'ansiedad': 579}
[INFO] Distribución val: {'depresion': 456, 'ansiedad': 185}

  RECORDATORIO: Estos splits eliminan leakage (pacientes disjuntos)
   Métricas serán más conservadoras pero generalizan mejor.

[INFO] Split aplicado (patient-level):
  Train: 1849 casos
  Val:   641 casos

[INFO] Distribución train: {'depresion': 1270, 'ansiedad': 579}
[INFO] Distribución val: {'depresion': 456, 'ansiedad': 185}

  RECORDATORIO: Estos splits eliminan leakage (pacientes disjuntos)
   Métricas serán más conservadoras pero generalizan mejor.


## 2) Split estratificado y entrenamiento (char TF‑IDF + LinearSVC)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

tfidf_char = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(3,5),
    min_df=2,
    max_df=0.95
)

clf = Pipeline([
    ('tfidf', tfidf_char),
    ('svm', LinearSVC(class_weight='balanced', random_state=42))
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print("[INFO] Entrenamiento completado.")

[INFO] Entrenamiento completado.


## 3) Métricas y exportables

**⚠️ IMPORTANTE - MANEJO DE CASOS NEUTRALES (TF-IDF vs Rule-Based):**

**TF-IDF es un modelo ML BINARIO FORZADO:**
- LinearSVC genera **solo 2 salidas posibles**: ansiedad o depresión
- **NO puede abstenerse** ni generar predicciones "neutral"
- Siempre clasifica cada caso en una de las dos clases (decisión forzada)

**Diferencia con Rule-Based:**
- **Rule-Based:** Puede devolver "neutral" (~78.4% casos sin matches)
  - Tiene 3 salidas: ansiedad, depresión, neutral
  - Para comparar con TF-IDF, convierte neutrales → mayoritaria
  
- **TF-IDF:** Siempre binario (0% neutrales)
  - Tiene 2 salidas: ansiedad o depresión
  - Aprende frontera de decisión en espacio char n-grams
  - Incluso casos ambiguos son forzados a una clase

**Implicaciones para comparación:**
1. ✅ TF-IDF comparable directamente con Dummy/BETO (todos binarios)
2. ⚠️ Comparación con Rule-Based es INJUSTA:
   - F1 RB bajo = 78% neutrales + errores en 22% detectado
   - F1 TF-IDF alto = decisión forzada en 100% casos
3. 📊 Gap real: TF-IDF aprende vocabulario paraguayo que RB no cubre

**En este notebook:**
- NO hay conversión de neutrales (modelo binario puro)
- Métricas reflejan capacidad discriminativa directa

In [4]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix

tfidf_pred_csv   = DATA_PATH/'tfidf_predictions.csv'
tfidf_report_csv = DATA_PATH/'tfidf_classification_report.csv'
tfidf_eval_csv   = DATA_PATH/'tfidf_eval.csv'
tfidf_cm_csv     = DATA_PATH/'tfidf_confusion_matrix.csv'

classes = ['depresion','ansiedad']

pd.DataFrame(classification_report(y_val, y_pred, labels=classes, output_dict=True, zero_division=0))  .transpose().to_csv(tfidf_report_csv, index=True, encoding='utf-8')

pd.DataFrame([{
    'macro_f1': f1_score(y_val, y_pred, average='macro', zero_division=0),
    'macro_precision': precision_score(y_val, y_pred, average='macro', zero_division=0),
    'macro_recall': recall_score(y_val, y_pred, average='macro', zero_division=0),
    'n': int(len(y_val))
}]).to_csv(tfidf_eval_csv, index=False, encoding='utf-8')

cm = confusion_matrix(y_val, y_pred, labels=classes)
pd.DataFrame(cm, index=[f'true_{c}' for c in classes], columns=[f'pred_{c}' for c in classes]).to_csv(tfidf_cm_csv)

pd.DataFrame({'texto': X_val, 'y_true': y_val, 'y_pred': y_pred}).to_csv(tfidf_pred_csv, index=False, encoding='utf-8')

print("[INFO] Exportados:")
print("  - Predicciones:", tfidf_pred_csv)
print("  - Reporte:", tfidf_report_csv)
print("  - Métricas:", tfidf_eval_csv)
print("  - Matriz:", tfidf_cm_csv)

[INFO] Exportados:
  - Predicciones: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_predictions.csv
  - Reporte: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_classification_report.csv
  - Métricas: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_eval.csv
  - Matriz: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/tfidf_confusion_matrix.csv


## 4) Análisis de Errores (FP/FN)

In [5]:
# Exportar errores para análisis cualitativo
# FP = predijo X cuando era Y (falsos positivos de cada clase)
# FN = predijo Y cuando era X (falsos negativos de cada clase)

# False Positives: casos predichos como depresión cuando eran ansiedad
fp_depresion = df_val[(y_val == 'ansiedad') & (y_pred == 'depresion')].copy()
fp_depresion['error_type'] = 'FP_depresion'

# False Negatives: casos que eran depresión pero se predijeron como ansiedad
fn_depresion = df_val[(y_val == 'depresion') & (y_pred == 'ansiedad')].copy()
fn_depresion['error_type'] = 'FN_depresion'

# False Positives: casos predichos como ansiedad cuando eran depresión
fp_ansiedad = df_val[(y_val == 'depresion') & (y_pred == 'ansiedad')].copy()
fp_ansiedad['error_type'] = 'FP_ansiedad'

# False Negatives: casos que eran ansiedad pero se predijeron como depresión  
fn_ansiedad = df_val[(y_val == 'ansiedad') & (y_pred == 'depresion')].copy()
fn_ansiedad['error_type'] = 'FN_ansiedad'

# Guardar análisis de errores
tfidf_fp_dep_csv = DATA_PATH / 'tfidf_fp_depresion.csv'
tfidf_fn_dep_csv = DATA_PATH / 'tfidf_fn_depresion.csv'
tfidf_fp_ans_csv = DATA_PATH / 'tfidf_fp_ansiedad.csv'
tfidf_fn_ans_csv = DATA_PATH / 'tfidf_fn_ansiedad.csv'

fp_depresion[[text_col, label_col, 'error_type']].to_csv(tfidf_fp_dep_csv, index=False, encoding='utf-8')
fn_depresion[[text_col, label_col, 'error_type']].to_csv(tfidf_fn_dep_csv, index=False, encoding='utf-8')
fp_ansiedad[[text_col, label_col, 'error_type']].to_csv(tfidf_fp_ans_csv, index=False, encoding='utf-8')
fn_ansiedad[[text_col, label_col, 'error_type']].to_csv(tfidf_fn_ans_csv, index=False, encoding='utf-8')

print("[INFO] Análisis de errores exportado:")
print(f"  FP Depresión: {len(fp_depresion)} casos → {tfidf_fp_dep_csv.name}")
print(f"  FN Depresión: {len(fn_depresion)} casos → {tfidf_fn_dep_csv.name}")
print(f"  FP Ansiedad:  {len(fp_ansiedad)} casos → {tfidf_fp_ans_csv.name}")
print(f"  FN Ansiedad:  {len(fn_ansiedad)} casos → {tfidf_fn_ans_csv.name}")

[INFO] Análisis de errores exportado:
  FP Depresión: 42 casos → tfidf_fp_depresion.csv
  FN Depresión: 27 casos → tfidf_fn_depresion.csv
  FP Ansiedad:  27 casos → tfidf_fp_ansiedad.csv
  FN Ansiedad:  42 casos → tfidf_fn_ansiedad.csv


## 5) Cross-Validation 5-Fold (Evaluación Robusta)

**Estrategia final:** Evaluar con CV para estimar F1 con intervalos de confianza.

**Justificación:**
- Dataset pequeño (90 pacientes) → alta varianza por muestreo
- Single test set (18 pacientes) puede variar ±10-15% por azar
- CV 5-fold usa TODOS los datos → estimación más confiable
- Permite reportar: F1 = 0.85 ± 0.03 (IC95%)

**Resultados previos:**
- Dev 60/20/20: F1 = 0.866 (18 pacientes)
- Test 60/20/20: F1 = 0.786 (18 pacientes)
- **Varianza observada:** 8 puntos de F1 entre evaluaciones

**Objetivo:** Estimar F1 real con todos los pacientes y cuantificar varianza.

---

**⚠️ MANEJO DE NEUTRALES EN CV:**

**TF-IDF NO genera predicciones neutrales en ningún fold:**
- Modelo ML binario forzado (LinearSVC con 2 clases)
- Varianza de CV refleja **solo capacidad de generalización**
- NO hay conversión de neutrales (contrario a Rule-Based)

**Diferencia con Rule-Based CV:**
- **Rule-Based:** ~78% neutrales por fold → convertidos a mayoritaria
  - Varianza CV = heterogeneidad dataset + cobertura variable
  - F1 penalizado por falta de cobertura
  
- **TF-IDF:** 0% neutrales (binario puro)
  - Varianza CV = capacidad de generalizar entre folds
  - F1 refleja discriminación real (no penalización por cobertura)

**Interpretación:**
- Si TF-IDF tiene menor varianza que RB → aprende mejor
- Si F1 TF-IDF >> F1 RB → vocabulario ML cubre dataset completo

In [6]:
from sklearn.model_selection import StratifiedKFold

print("="*80)
print("CROSS-VALIDATION 5-FOLD (PATIENT-LEVEL)")
print("="*80)
print()

# Configuración
N_SPLITS = 5
RANDOM_STATE = 42

# Preparar dataset con row_id
df_full = dataset_base.copy()
if 'row_id' not in df_full.columns:
    df_full = df_full.reset_index(drop=True)
    df_full['row_id'] = df_full.index

# Limpiar NaNs
df_full = df_full.dropna(subset=['texto', label_col]).copy()
df_full['texto_ml'] = df_full['texto'].map(clean_text_ml)

print(f"✓ Dataset completo: {len(df_full)} casos")
print(f"✓ Pacientes únicos: {df_full['patient_id'].nunique()}")
print()

# Obtener etiqueta mayoritaria por paciente (para stratification)
patient_labels = df_full.groupby('patient_id')[label_col].agg(
    lambda x: x.value_counts().index[0]
).reset_index()
patient_labels.columns = ['patient_id', 'label_majority']

# Crear folds stratificados
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
patient_ids = patient_labels['patient_id'].values
patient_y = patient_labels['label_majority'].values

# Ejecutar CV
cv_results = []

for fold_idx, (train_patient_idx, test_patient_idx) in enumerate(skf.split(patient_ids, patient_y), start=1):
    print(f"Fold {fold_idx}/{N_SPLITS}...", end=" ")
    
    # Obtener pacientes
    train_patients = patient_ids[train_patient_idx]
    test_patients = patient_ids[test_patient_idx]
    
    # Filtrar casos
    train_df = df_full[df_full['patient_id'].isin(train_patients)]
    test_df = df_full[df_full['patient_id'].isin(test_patients)]
    
    X_train_cv = train_df['texto_ml']
    y_train_cv = train_df[label_col]
    X_test_cv = test_df['texto_ml']
    y_test_cv = test_df[label_col]
    
    # Entrenar modelo (mismo pipeline que antes)
    clf_cv = Pipeline([
        ('tfidf', TfidfVectorizer(
            analyzer='char_wb',
            ngram_range=(3, 5),
            min_df=2,
            max_df=0.95
        )),
        ('svm', LinearSVC(class_weight='balanced', random_state=42, max_iter=2000))
    ])
    
    clf_cv.fit(X_train_cv, y_train_cv)
    y_pred_cv = clf_cv.predict(X_test_cv)
    
    # Métricas
    f1_cv = f1_score(y_test_cv, y_pred_cv, average='macro', zero_division=0)
    prec_cv = precision_score(y_test_cv, y_pred_cv, average='macro', zero_division=0)
    rec_cv = recall_score(y_test_cv, y_pred_cv, average='macro', zero_division=0)
    
    cv_results.append({
        'fold': fold_idx,
        'f1_macro': f1_cv,
        'precision': prec_cv,
        'recall': rec_cv,
        'n_train_patients': len(train_patients),
        'n_test_patients': len(test_patients),
        'n_train_cases': len(X_train_cv),
        'n_test_cases': len(X_test_cv)
    })
    
    print(f"F1={f1_cv:.3f}, Prec={prec_cv:.3f}, Rec={rec_cv:.3f}")

# Resultados
cv_df = pd.DataFrame(cv_results)

print()
print("="*80)
print("RESULTADOS CROSS-VALIDATION")
print("="*80)
print()
print(cv_df[['fold', 'f1_macro', 'precision', 'recall', 'n_test_patients']].to_string(index=False))
print()

# Estadísticas
f1_mean = cv_df['f1_macro'].mean()
f1_std = cv_df['f1_macro'].std()
f1_min = cv_df['f1_macro'].min()
f1_max = cv_df['f1_macro'].max()
f1_ci_lower = f1_mean - 1.96 * f1_std
f1_ci_upper = f1_mean + 1.96 * f1_std

print(f"📊 ESTADÍSTICAS:") 
print(f"   F1 macro:  {f1_mean:.3f} ± {f1_std:.3f}")
print(f"   Precision: {cv_df['precision'].mean():.3f} ± {cv_df['precision'].std():.3f}")
print(f"   Recall:    {cv_df['recall'].mean():.3f} ± {cv_df['recall'].std():.3f}")
print()
print(f"   F1 min-max: [{f1_min:.3f}, {f1_max:.3f}]")
print(f"   F1 IC95%:   [{f1_ci_lower:.3f}, {f1_ci_upper:.3f}]")
print(f"   Varianza:   {(f1_max - f1_min):.3f} puntos entre folds")
print()

# Interpretación
print("🔍 INTERPRETACIÓN:")
print(f"   • F1 varía {(f1_max - f1_min):.3f} puntos entre folds → normal en dataset pequeño")
print(f"   • IC95%: [{f1_ci_lower:.3f}, {f1_ci_upper:.3f}] → rango esperado para nuevos pacientes")
print(f"   • TF-IDF generaliza bien (modelo binario forzado, 0% neutrales)")

print()
print(f"📈 CONCLUSIÓN:")
print(f"   • F1 TF-IDF (CV): {f1_mean:.3f} ± {f1_std:.3f}")
print(f"   • IC95%: [{f1_ci_lower:.3f}, {f1_ci_upper:.3f}]")
print(f"   • Varianza: {(f1_max - f1_min):.3f} puntos (normal en 90 pacientes)")
print(f"   • Comparación: Dummy Stratified (0.50) → TF-IDF (0.85) = +70% mejora")
print()

# Exportar
cv_output = DATA_PATH / 'cv_results' / 'tfidf_cv_results.csv'
cv_output.parent.mkdir(exist_ok=True)
cv_df.to_csv(cv_output, index=False)
print(f"💾 Resultados exportados: {cv_output}")
print()
print("="*80)
print("✅ Cross-Validation completado")
print("="*80)

CROSS-VALIDATION 5-FOLD (PATIENT-LEVEL)

✓ Dataset completo: 3126 casos
✓ Pacientes únicos: 90

Fold 1/5... ✓ Dataset completo: 3126 casos
✓ Pacientes únicos: 90

Fold 1/5... F1=0.801, Prec=0.817, Rec=0.790
Fold 2/5... F1=0.801, Prec=0.817, Rec=0.790
Fold 2/5... F1=0.867, Prec=0.876, Rec=0.859
Fold 3/5... F1=0.867, Prec=0.876, Rec=0.859
Fold 3/5... F1=0.843, Prec=0.831, Rec=0.866
Fold 4/5... F1=0.843, Prec=0.831, Rec=0.866
Fold 4/5... F1=0.883, Prec=0.904, Rec=0.868
Fold 5/5... F1=0.883, Prec=0.904, Rec=0.868
Fold 5/5... F1=0.854, Prec=0.838, Rec=0.878

RESULTADOS CROSS-VALIDATION

 fold  f1_macro  precision   recall  n_test_patients
    1  0.801422   0.817126 0.789808               18
    2  0.866665   0.875965 0.859445               18
    3  0.842912   0.831371 0.866074               18
    4  0.883363   0.903846 0.867605               18
    5  0.853742   0.838419 0.877597               18

📊 ESTADÍSTICAS:
   F1 macro:  0.850 ± 0.031
   Precision: 0.853 ± 0.036
   Recall:    0.852 

## 6) Exportar Resultados y Próximos Pasos

**✅ Archivos generados por este baseline:**

Evaluación en dev set:
- `tfidf_predictions.csv` - Predicciones por caso
- `tfidf_eval.csv` - Métricas macro agregadas
- `tfidf_classification_report.csv` - Reporte por clase
- `tfidf_confusion_matrix.csv` - Matriz de confusión

Cross-Validation:
- `cv_results/tfidf_cv_results.csv` - Resultados 5-fold CV

---

**📊 Para análisis comparativo completo:**
→ Ejecutar notebook: `02_comparacion_resultados.ipynb`

Este notebook consolida todos los resultados CV, calcula estadísticas (IC95%), compara modelos, y genera visualizaciones e interpretación para paper/tesis.

---

**📝 Notas metodológicas:**
- **Dataset:** dataset_base.csv (3,155 casos, 90 pacientes)
- **Split:** Patient-level 60/20/20 (0% leakage)
- **CV:** 5-fold patient-level stratified (54 pacientes train por fold)
- **Modelo:** TfidfVectorizer char(3-5) + LinearSVC(C=1.0, class_weight='balanced')
- **Preprocesamiento:** Agresivo (lowercase, sin tildes, sin puntuación)